In [1]:
import os
import pandas as pd

from rdkit import Chem # safe import before ccdc imports
from torch_geometric.loader import DataLoader
from torch.utils.data import ConcatDataset

from conf_ensemble_dataset_in_memory import ConfEnsembleDataset
from litschnet import LitSchNet
from molsize_model import MolSizeModel
from rmsd_predictor_evaluator import RMSDPredictorEvaluator
from tqdm import tqdm

# Data preparation

In [2]:
# run once to preprocess datasets and generate chunks
dataset = ConfEnsembleDataset()
# dataset = ConfEnsembleDataset(dataset='platinum') # 16G

In [3]:
def get_test_dataset(split, 
                    iteration, 
                    data_dir='data/',
                    ) :
    
    pdbbind_chunks = [filename for filename in os.listdir(os.path.join(data_dir, 'processed')) if filename.startswith('pdbbind')]
    pdbbind_n_chunks = len(pdbbind_chunks)
    
    test_datasets = []
    
    if split in ['random', 'scaffold'] :

        with open(os.path.join(data_dir, f'ligand_{split}_splits', f'train_smiles_{split}_split_{iteration}.txt'), 'r') as f :
            train_smiles = f.readlines()
            train_smiles = [smiles.strip() for smiles in train_smiles]
        
        with open(os.path.join(data_dir, f'ligand_{split}_splits', f'test_smiles_{split}_split_{iteration}.txt'), 'r') as f :
            test_smiles = f.readlines()
            test_smiles = [smiles.strip() for smiles in test_smiles]

        for chunk_number in tqdm(range(pdbbind_n_chunks)) :

            dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                          smiles_list=test_smiles)
            test_datasets.append(dataset)
            
    else : #protein split

        with open(os.path.join(data_dir, 'protein_similarity_splits', f'train_pdb_protein_similarity_split_{iteration}.txt'), 'r') as f :
            train_pdbs = f.readlines()
            train_pdbs = [pdb.strip() for pdb in train_pdbs]
        
        with open(os.path.join(data_dir, 'protein_similarity_splits', f'test_pdb_protein_similarity_split_{iteration}.txt'), 'r') as f :
            test_pdbs = f.readlines()
            test_pdbs = [pdb.strip() for pdb in test_pdbs]

        smiles_df = pd.read_csv('data/smiles_df.csv')
        train_smiles = smiles_df[smiles_df['id'].isin(train_pdbs)]['smiles'].values
            
        for chunk_number in tqdm(range(pdbbind_n_chunks)) :

            dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                          pdb_ids_list=test_pdbs)
            test_datasets.append(dataset)

    test_dataset = ConcatDataset(test_datasets)
    
    return test_dataset, train_smiles

In [4]:
def evaluate_model(experiment_name,
                   test_dataset,
                   #platinum_dataset,
                   training_smiles,
                   tasks = ['all', 'easy', 'hard']) :
    
    checkpoint_name = os.listdir(os.path.join('lightning_logs', experiment_name, 'checkpoints'))[0]
    checkpoint_path = os.path.join('lightning_logs', experiment_name, 'checkpoints', checkpoint_name)
    if 'molsize' in experiment_name :
        model = MolSizeModel.load_from_checkpoint(checkpoint_path=checkpoint_path)
    else :
        model = LitSchNet.load_from_checkpoint(checkpoint_path=checkpoint_path)
    
    evaluation_name = experiment_name + '_pdbbind'
    evaluator = RMSDPredictorEvaluator(model=model, 
                                       evaluation_name=evaluation_name,
                                       training_smiles=train_smiles)
    #evaluator.evaluate(test_dataset)
    evaluator.evaluate(test_dataset, overwrite=True)
    for task in tasks :
        evaluator.evaluation_report(task=task)

#     evaluation_name = experiment_name + '_platinum'
#     evaluator = RMSDPredictorEvaluator(model=model, 
#                                        evaluation_name=evaluation_name,
#                                        training_smiles=train_smiles)
#     evaluator.evaluate(platinum_dataset, overwrite=True)
#     for task in tasks :
#         evaluator.evaluation_report(task=task)

In [5]:
data_dir = 'data/'
# platinum_chunks = [filename for filename in os.listdir(os.path.join(data_dir, 'processed')) if filename.startswith('platinum')]
# platinum_n_chunks = len(platinum_chunks)

In [6]:
# platinum_datasets = []
# for chunk_number in tqdm(range(platinum_n_chunks)) :
#     dataset = ConfEnsembleDataset(dataset='platinum', loaded_chunk=chunk_number)
#     platinum_datasets.append(dataset)
# platinum_dataset = ConcatDataset(platinum_datasets)

In [7]:
splits = ['random', 'scaffold', 'protein']

In [ ]:
%%time

#for split in splits :
for split in ['protein'] :

    for iteration in range(5) :

        test_dataset, train_smiles = get_test_dataset(split, iteration)
        
        experiment_name = f'{split}_split_{iteration}'
        # evaluate_model(experiment_name, test_dataset, platinum_dataset, train_smiles)
        evaluate_model(experiment_name, test_dataset, train_smiles)
        
        experiment_name = f'{split}_split_{iteration}_molsize'
        evaluate_model(experiment_name, test_dataset, train_smiles)

100%|█████████████████████████████████████████████| 3/3 [01:44<00:00, 34.88s/it]


Computing training set fingerprints
Grouping data by smiles


  0%|                                                  | 0/1412 [00:00<?, ?it/s]

Starting evaluation


100%|███████████████████████████████████████| 1412/1412 [01:24<00:00, 16.76it/s]


...

100%|███████████████████████████████████████| 1429/1429 [01:10<00:00, 20.20it/s]
/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/numpy/core/_methods.p

Computing training set fingerprints
Grouping data by smiles


  0%|                                          | 2/1373 [00:00<01:54, 11.93it/s]

Starting evaluation


100%|███████████████████████████████████████| 1373/1373 [01:31<00:00, 14.99it/s]


In [10]:
len(test_dataset)

101506

In [12]:
import importlib
import rmsd_predictor_evaluator
importlib.reload(rmsd_predictor_evaluator)
RMSDPredictorEvaluator = rmsd_predictor_evaluator.RMSDPredictorEvaluator

In [13]:
experiment_name = f'{split}_split_{iteration}'
# evaluate_model(experiment_name, test_dataset, platinum_dataset, train_smiles)
evaluate_model(experiment_name, test_dataset, train_smiles)

Computing training set fingerprints
Grouping data by smiles


  0%|                                          | 1/1418 [00:00<03:16,  7.22it/s]

Starting evaluation


  3%|█▏                                       | 40/1418 [00:03<01:42, 13.38it/s]

> /home/benoit/bioactive_conformation_predictor/rmsd_predictor_evaluator.py(260)mol_evaluation()
    258 
    259             # Generated stats
--> 260             generated_targets = mol_targets[is_generated]
    261             generated_preds = mol_preds[is_generated]
    262             generated_energies = mol_energies[is_generated]



...

ipdb>  len([d.data_id for d in smiles_data_list])


100


ipdb>  exit


  3%|█                                     | 40/1418 [25:02<14:22:36, 37.56s/it]


BdbQuit: 